<a href="https://colab.research.google.com/github/seein2/capstone/blob/seein/python/model_practice/KcELECTRA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install datasets
!pip install pandas
!pip install torch
!pip install transformers
!pip install openpyxl
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import AdamW, get_scheduler
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
# KcELECTRA 모델과 토크나이저 불러오기
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/450k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data = pd.read_csv('./drive/MyDrive/final_dataset.csv')
data

,Unnamed: 0,발화,감정
0,0,언니 동생으로 부르는게 맞는 일인가요..??,불안
1,1,그냥 내 느낌일뿐겠지?,불안
2,2,아직너무초기라서 그런거죠?,불안
3,3,유치원버스 사고 낫다던데,불안
4,4,근데 원래이런거맞나요,불안
...,...,...,...
90962,58266,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,당황
90963,58267,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아...,당황
90964,58268,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.요...,당황
90965,58269,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,당황


In [ ]:
# 라벨링
data.loc[(data['감정'] == "불안"), '감정'] = 0  #불안 => 0
data.loc[(data['감정'] == "당황"), '감정'] = 1  #당황 => 1
data.loc[(data['감정'] == "분노"), '감정'] = 2  #분노 => 2
data.loc[(data['감정'] == "슬픔"), '감정'] = 3  #슬픔 => 3
data.loc[(data['감정'] == "중립"), '감정'] = 4  #중립 => 4
data.loc[(data['감정'] == "행복"), '감정'] = 5  #행복 => 5

data_list = []
for ques, label in zip(data['발화'], data['감정']):
    data = []
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=2000)
print(len(dataset_train), len(dataset_test))

68225 22742


In [ ]:
# 데이터셋 클래스 정의
class EmotionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence, label = self.data[idx]
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        label = torch.tensor(int(label), dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': label
        }

In [ ]:
# 학습 데이터셋과 테스트 데이터셋 준비
train_dataset = EmotionDataset(dataset_train, tokenizer)
test_dataset = EmotionDataset(dataset_test, tokenizer)

In [ ]:
# DataLoader 정의 (배치 단위로 데이터를 불러오기 위함)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
# 체크포인트를 저장할 폴더 생성
checkpoint_dir = "./drive/MyDrive/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# 체크포인트 저장 함수
def save_checkpoint(epoch, model, optimizer, loss, checkpoint_dir, filename='checkpoint.pth'):
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"Checkpoint saved: {checkpoint_path}")

In [ ]:
# 옵티마이저 정의
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프 정의
epochs = 8

# 학습률 스케줄러 정의
num_training_steps = epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for epoch in range(epochs):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0
    total_correct = 0
    total_examples = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()

        # 데이터 준비
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # 모델에 입력 데이터 전달 후 출력 획득
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # 역전파 및 옵티마이저 스텝
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()

        # 예측 값과 실제 레이블을 비교하여 정확도 계산
        preds = torch.argmax(logits, dim=-1)
        total_correct += (preds == labels).sum().item()
        total_examples += labels.size(0)

    avg_train_loss = total_loss / len(train_dataloader)
    accuracy = total_correct / total_examples

    print(f"Epoch {epoch + 1}, Loss: {avg_train_loss}, Accuracy: {accuracy:.4f}")
    # 체크포인트 저장
    save_checkpoint(epoch + 1, model, optimizer, avg_train_loss, checkpoint_dir)

100%|██████████| 1067/1067 [05:37<00:00,  3.16it/s]


Epoch 1, Loss: 0.8288710477649328, Accuracy: 0.6931
Checkpoint saved: ./drive/MyDrive/checkpoints/checkpoint_epoch_1.pth


100%|██████████| 1067/1067 [05:36<00:00,  3.17it/s]


Epoch 2, Loss: 0.6129540238449143, Accuracy: 0.7799
Checkpoint saved: ./drive/MyDrive/checkpoints/checkpoint_epoch_2.pth


100%|██████████| 1067/1067 [05:36<00:00,  3.18it/s]


...

Epoch 6, Loss: 0.11645074592747891, Accuracy: 0.9638
Checkpoint saved: ./drive/MyDrive/checkpoints/checkpoint_epoch_6.pth


100%|██████████| 1067/1067 [05:36<00:00,  3.17it/s]


Epoch 7, Loss: 0.07012395360249582, Accuracy: 0.9793
Checkpoint saved: ./drive/MyDrive/checkpoints/checkpoint_epoch_7.pth


100%|██████████| 1067/1067 [05:36<00:00,  3.18it/s]


Epoch 8, Loss: 0.04553199475706401, Accuracy: 0.9873
Checkpoint saved: ./drive/MyDrive/checkpoints/checkpoint_epoch_8.pth


In [ ]:
model.eval()  # 평가 모드로 설정
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy}")

100%|██████████| 356/356 [00:40<00:00,  8.76it/s]

Test Accuracy: 0.7384574795532495


In [ ]:
def analyze_diary(model, tokenizer, diary_text):
    model.eval()

    # 문장 분할 (마침표, 느낌표, 물음표, 세미콜론, 쉼표로 문장을 구분)
    sentences = re.split(r'(?<=[.!?,;])\s*', diary_text.strip())

    # 감정 목록 및 확률 초기화
    emotions = ["불안", "당황", "분노", "슬픔", "중립", "행복"]
    total_probabilities = np.zeros(len(emotions))  # 감정별 총합 확률

    # 문장별 감정 분석
    for idx, sentence in enumerate(sentences):
        if sentence:  # 빈 문장은 제외
            print(f"\nAnalyzing Sentence {idx + 1}: {sentence}")

            encoding = tokenizer.encode_plus(
                sentence,
                add_special_tokens=True,
                max_length=128,
                return_attention_mask=True,
                return_tensors='pt',
                padding='max_length',
                truncation=True
            )

            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)

            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask)

            # 문장별 감정 확률 계산
            probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
            intensity = probabilities.squeeze().cpu().numpy()

            # 감정 확률을 총합에 더함
            total_probabilities += intensity

    # 중립(4번째 감정)을 제외한 감정 확률을 계산
    non_neutral_emotions = [(emotions[i], total_probabilities[i]) for i in range(len(emotions)) if i != 4]

    # 감정 확률을 정규화하여 100%로 만듦
    non_neutral_probabilities = [prob for _, prob in non_neutral_emotions]
    normalized_probabilities = np.array(non_neutral_probabilities) / sum(non_neutral_probabilities) * 100

    # 상위 3개의 감정을 추출(5퍼센트 미만 감정은 제외)
    top_3_indices = np.argsort(normalized_probabilities)[-3:][::-1]
    top_3_emotions = [(non_neutral_emotions[i][0], normalized_probabilities[i]) for i in top_3_indices if normalized_probabilities[i] >= 5]

    # 결과 출력
    print("\nOverall Top 3 Emotion Analysis:")
    for emotion, prob in top_3_emotions:
        print(f"{emotion}: {prob:.2f}%")

In [ ]:
# 사용자가 입력한 일기 예시
diary_text = "오빠안녕 난 지금 수업을 듣고있고있어. 근데 밖에 날씨를 보니까 엄청 흐려ㅠㅠ 곧 비가 올거같아 오빠는 우산이 있지만 나는 없어 어떡하지?오빠가 데리러와야되나ㅠㅠ"
analyze_diary(model, tokenizer, diary_text)


Analyzing Sentence 1: 오빠안녕 난 지금 수업을 듣고있고있어.

Analyzing Sentence 2: 근데 밖에 날씨를 보니까 엄청 흐려ㅠㅠ 곧 비가 올거같아 오빠는 우산이 있지만 나는 없어 어떡하지?

Analyzing Sentence 3: 오빠가 데리러와야되나ㅠㅠ

Overall Top 3 Emotion Analysis:
슬픔: 60.85%
행복: 33.30%
불안: 5.69%
